### Fase 5: Creación de una Base de Datos
Organiza toda la información recopilada en una base de datos SQL bien estructurada. Define las tablas y relaciones necesarias para almacenar los datos de manera eficiente.


### Fase 6: Inserción de Datos en la Base de Datos
Inserta todos los datos recopilados en la base de datos diseñada.

___

In [60]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sys.path.append("../src")
import funciones_bbdd as fb

In [61]:
nombre_bbdd = "pruba_data"
conexion = fb.conexion_bbdd(nombre_bbdd)
cursor = conexion.cursor()

In [62]:
fb.creacion_tablas(conexion, cursor)

In [89]:
df_peliculas = pd.read_csv("../datos/datos_ana/peliculas.csv")
df_detalles_peliculas = pd.read_csv("../datos/datos_ana/detalles_peliculas.csv")
df_pelicula_artista = pd.read_csv("../datos/datos_ana/pelicula_artista.csv")
df_detalles_artistas = pd.read_csv("../datos/datos_ana/detalles_artistas.csv")

Insertamos los géneros:

In [5]:
lista_generos = list(df_peliculas["generos"].unique())
lista_tuplas_super = [(x,) for x in lista_generos]
query_insercion = "insert into generos (nombre) values (%s)"
cursor.executemany(query_insercion, lista_tuplas_super)
conexion.commit()

In [6]:
lista_tuplas_super = [tuple(fila) for fila in df_detalles_artistas.values]
query_insercion = "insert into actores (nombre, anio, conocido_por, que_hace, premios) values (%s, %s, %s, %s, %s)"
cursor.executemany(query_insercion, lista_tuplas_super)
conexion.commit()

Antes de insertar la tabla de película sustituimos los géneros por sus id:

In [64]:
consulta = """SELECT *
FROM generos g ;"""

conexion = fb.conexion_bbdd(nombre_bbdd)
cursor = conexion.cursor()
cursor.execute(consulta)
data = cursor.fetchall()
data

[(1, 'Action'),
 (2, 'Drama'),
 (3, 'Comedy'),
 (4, 'Fantasy'),
 (5, 'Horror'),
 (6, 'Romance'),
 (7, 'Thriller'),
 (8, 'Mystery')]

In [65]:
dic_map = {}
for t in data:
    dic_map[t[1]]=t[0]

df_peliculas["generos"] = df_peliculas["generos"].map(dic_map)

Mergemamos las tablas para tener toda la información de una misma película en una única fila:

In [66]:
df_merged = df_peliculas.merge(df_detalles_peliculas, left_on="Id", right_on="id_pelicula")
df_merged.drop(columns=["id_pelicula", "titulo"], inplace=True)
df_merged.head(1)

,Tipo,Titulo,Año,Mes,Id,generos,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd
0,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,1,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min


In [67]:
df_merged["cali_imdb"] = df_merged["cali_imdb"].fillna(0)

Reordenamos para insertar en la tabla en el mismo orden que se crearon las columnas:

In [69]:
df_ordenado = df_merged.iloc[:, [4, 1, 0, 5, 2, 3, 6, 10, 7, 8, 9]]

Insertamos la tabla películas:

In [70]:
lista_tuplas_super = [tuple(fila) for fila in df_ordenado.values]
query_insercion = "insert into peliculas (id_pelicula, titulo, tipo, id_genero, anio, mes, nota, duracion, directores, guionistas, argumento) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(query_insercion, lista_tuplas_super)
conexion.commit()

[('tt0320656', 'Las Travesuras de Super Chido', 'Movie', 1, 1990, 10, 7.7, '1h 35min', 'Alejandro Todd', 'Óscar FentanesJuan Garrido', 'Desconocido')]


### Inserción tabla `Apariciones`

In [77]:
df_pelicula_artista[df_pelicula_artista["Actriz_Actor"] != "Sin datos"]

,Movie ID,Actriz_Actor
0,tt0110660,Tony Ka Fai Leung
1,tt0110363,Luke Edwards
2,tt0110397,Jennifer Barker
3,tt0111045,Miklós Déri
4,tt0111653,Joe Bays
...,...,...
14504,tt10709060,Kendra Kuhnhofer
14514,tt14142640,Prakash
14519,tt18688348,Oleg Primogenov
14522,tt27241513,Georgiy Topolaga


In [81]:
consulta = """SELECT id_actor , nombre 
                FROM actores a ;"""

conexion = fb.conexion_bbdd(nombre_bbdd)
cursor = conexion.cursor()
cursor.execute(consulta)
data = cursor.fetchall()

In [ ]:
dic_id_actor = {}
for t in data:
    dic_id_actor[t[1].lower()]=t[0]


df_pelicula_artista["Actriz_Actor"]=df_pelicula_artista["Actriz_Actor"].str.lower()

df_pelicula_artista["Actriz_Actor"] = df_pelicula_artista["Actriz_Actor"].map(dic_map)

In [95]:
df_pelicula_artista[df_pelicula_artista["Actriz_Actor"].notna()]

,Movie ID,Actriz_Actor


Al parecer no hay ningún actor en la tabla df_pelicula_artista igual en nombre a los actores de la tabla df_detalles_artistas, por lo que no tiene ningún sentido crear esta última tabla.